# This version uses the same logic as in v_10 but uses one week as its timeframe

#### Have also updated the FCR and aFRR functions to make them faster and make FCR handle missing values

In [2]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, analysis, timeframes
import numpy as np
import calendar 
from datetime import datetime
import pytz
import openpyxl
import os


In [4]:
timeframe = timeframes.one_day
L, M, F, H, freq_data, power_meter_dict, consumption_data = utils.get_all_sets(timeframe)

In [6]:
L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m = utils.get_parameters(L,M,H)

In [7]:

markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())
price_list = [market.price_data for market in M]
volume_list = [market.volume_data for market in M]


In [8]:
print(f"Amount of markets : {len(M)}")
print(f"Amount of meters : {len(L)}")
print(f"Amount of meters with direction up or both : {len(L_u)}")
print(f"Amount of meters with direction down or both : {len(L_d)}")
print(f"Amount of hours : {len(H)}")

Amount of markets : 62
Amount of meters : 2189
Amount of meters with direction up or both : 2035
Amount of meters with direction down or both : 2038
Amount of hours : 24


In [9]:
total_up_flex = np.sum(Fu_h_l) # total available flex volume up
total_down_flex = np.sum(Fd_h_l) # total available flex volume down
total_response_time = np.sum(R_h_l) # total response time
#total_flex = total_up_flex + total_down_flex
average_response_time = total_response_time/ (len(H)*len(L))
hourly_flex_up = total_up_flex/len(H)
hourly_flex_down = total_down_flex/len(H)

print(f"Total up flex volume: {total_up_flex} MW")
print(f"Total down flex volume: {total_down_flex} MW")
print(f"Average flex volume pr hour up: {hourly_flex_up} MWh")
print(f"Average flex volume pr hour down: {hourly_flex_down} MWh")
print(f"Average response time: {average_response_time} seconds")

Total up flex volume: 148.49677599999998 MW
Total down flex volume: 150.634953 MW
Average flex volume pr hour up: 6.187365666666666 MWh
Average flex volume pr hour down: 6.2764563749999995 MWh
Average response time: 147.52632675429544 seconds


In [10]:
dominant_directions = [utils.get_dominant_direction(freq_data, hour) for hour in H]

In [12]:
Ir_hlm, Ia_hlm, Va_hm = utils.get_income_dictionaries(H, M, L, dominant_directions, Fu_h_l, Fd_h_l, P_h_m, Vp_h_m, F, markets_name_dict, timeframe)


Regulation Up Activated


/Users/sandermeland/Documents/Dokumenter – Sanders MacBook Air/UIB/Master/master-kode/code_map/utils.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)


Regulation Down Activated


/Users/sandermeland/Documents/Dokumenter – Sanders MacBook Air/UIB/Master/master-kode/code_map/utils.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)


In [13]:
compatible_list = utils.get_compatibility_dict(L, M)

In [14]:
# Create a new model
test_model = gp.Model("AssetToMarket")

# Create decision variables
x = {}
d = {}
y = {}
w = {}
for h in range(len(H)):
    for l in range(len(L)):
        for m in range(len(M)):
            # asset i is connected to market j at hour h
            x[h, l, m] = test_model.addVar(lb = 0, ub = 1, vtype=gp.GRB.BINARY, name=f"x_{h}_{l}_{m}")

            d[h,l,m] = 1 if l in compatible_list[m] else 0 # compatible_list takes care of both the area constraint and the direction constraint
            
            # adding the constraint
            test_model.addConstr(x[h,l,m] <= d[h,l,m])
    for m in range(len(M)):
        # market m has a bid at hour h
        y[h, m] = test_model.addVar(lb = 0, ub = 1, vtype=gp.GRB.BINARY, name=f"y_{h}_{m}")
        # market m is activated at hour h
        w[h, m] = test_model.addVar(lb = 0, ub = 1, vtype=gp.GRB.BINARY , name=f"w_{h}_{m}")
        

# Set objective            

# Set the objective to maximize the total income expression
test_model.setObjective(sum(Ir_hlm[h,l,m] * x[h,l,m] + Ia_hlm[h,l,m] * x[h,l,m] * w[h,m] for h in range(len(H)) for l in range(len(L)) for m in range(len(M)))
                        , gp.GRB.MAXIMIZE) # can possibly remove the x on the activation income

# Add constraints
for h in range(len(H)):
    for l in range(len(L)):
        # Each asset can only be connected to one market at a time
        test_model.addConstr(sum(x[h, l, m] for m in range(len(M))) <= 1, f"single_market_for_asset_at_hour_{h}_nr.{l}")
        

    for m, market in enumerate(M):
        
        up_val, down_val = F[h,m]
        if up_val + down_val > 0:
            test_model.addConstr(w[h,m] <= y[h,m], f"market_{m}_can_not_be_activated_at_hour_{h}_if_it_is_not_active")
        else:
            test_model.addConstr(w[h,m] == 0, f"market_{m}_can_not_be_activated_at_hour_{h}_if_it_is_not_active")
        
        
        # Connect the binary variables by using big M
        #test_model.addConstr(sum(x[h, l, m] for l in range(len(L))) <= len(L) * y[h, m], f"asset_connection_for_hour_{h}_market_{m}")
     
        # Max volume constraint
        
        if market.direction == "up":
            # capacity volume constraint
            test_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) <= Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
            # activation volume constraint
            test_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) * w[h,m] <= Va_hm[h,m], f"max_volume_for_activation_in-_market_{m}_at_hour_{h}")
            # min volume capacity constraint
            test_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") 

        elif market.direction == "down":
            # max capacity volume constraint
            test_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) <=  Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
            # max activation volume constraint
            test_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) * w[h,m] <= Va_hm[h,m], f"max_volume_for_activation_in_market_{m}_at_hour_{h}")
            # min volume capacity constraint
            test_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") 

        else: # market.direction == "both"
            if dominant_directions[h] == "up":
                # max capacity volume constraint
                test_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) <= Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
                # max activation volume constraint
                test_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) * w[h,m] <= Va_hm[h,m] , f"max_volume_for_activation_in_market_{m}_at_hour_{h}")
                # min capacity volume constraint
                test_model.addConstr(sum(x[h, l, m] * Fu_h_l[h,l] for l in range(len(L))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") 

            else:
                # max capacity volume constraint
                test_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) <= Vp_h_m[h,m] * y[h,m], f"max_volume_for_hour_{h}_market_{m}")
                # max activation volume constraint
                test_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) * w[h,m] <= Va_hm[h,m], f"max_volume_for_activation_in_market_{m}_at_hour_{h}")
                # min capacity volume constraint
                test_model.addConstr(sum(x[h, l, m] * Fd_h_l[h,l] for l in range(len(L))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") 

    
        
        # The response times for loads l connected to market m cannot exceed the max response time for m
        for l in range(len(L)):
            test_model.addConstr(x[h,l,m] * R_h_l[h,l] <= R_m[m] * y[h,m], f"response_time_for_hour_{h}_market_{m}")
        
            
            
            
# Enable logging
test_model.setParam('LogFile', 'weekly_test_model.log')

# Solve the model
test_model.optimize()
    
if test_model.status == gp.GRB.Status.INFEASIBLE:
    test_model.computeIIS()


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-18
Set parameter LogFile to value "weekly_test_model.log"
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6571464 rows, 3260208 columns and 16497577 nonzeros
Model fingerprint: 0x60504e45
Model has 132702 quadratic objective terms
Model has 1488 quadratic constraints
Variable types: 0 continuous, 3260208 integer (3260208 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+03]
  QMatrix range    [1e-06, 8e-01]
  Objective range  [5e-06, 2e+05]
  QObjective range [1e-06, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [2e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 6527326 rows and 1867366 columns (presolve time = 5s) ...
Presolve removed 6529308 rows and 3230436 columns (presolv

In [16]:
utils.test_solution_validity(x, y, w, Va_hm, L, M, H, dominant_directions= dominant_directions, F = F)

AssertionError: Minimum volume constraint violated at hour 7 for market 31

In [ ]:
import pickle

In [ ]:
# Extract binary variable values from the original model
new_x_values = {(h, l, m): test_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}


In [ ]:
# Load the saved values
with open('current_x_values_for_week.pkl', 'rb') as f:
    original_x_values = pickle.load(f)

old_dict = utils.get_market_count_dict(original_x_values)
new_dict = utils.get_market_count_dict(new_x_values)

differences = {}
for key in old_dict:
    if not old_dict[key].equals(new_dict[key]):
        differences[key] = (new_dict[key], old_dict[key])
        

for key, (orig_val, mod_val) in differences.items():
    print(f"Difference for hour {key}: \n Original={display(orig_val)}, \n  Modified={display(mod_val)}")


In [ ]:
# Extract binary variable values from the original model
current_x_values = {(h, l, m): test_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}

# Save these values
with open('current_x_values_for_week.pkl', 'wb') as f:
    pickle.dump(current_x_values, f)
                